In [3]:
import keras
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization,Conv2D,MaxPooling2D
import os
from keras.layers import AveragePooling2D

In [4]:
n=7
rows=48
cols=48
batch_size=105
train=r'C:\Users\majaa\OneDrive\Desktop\images\train'
validate=r'C:\Users\majaa\OneDrive\Desktop\images\validation'

In [5]:
train_data_gen=ImageDataGenerator(rescale=1./255,rotation_range=30,shear_range=0.3,zoom_range=0.3,width_shift_range=0.4,height_shift_range=0.4,horizontal_flip= True,fill_mode='nearest')

In [6]:
val_data_gen=ImageDataGenerator(rescale=1./255)

In [7]:
train_gen=train_data_gen.flow_from_directory(train,color_mode='grayscale',target_size=(rows,cols),batch_size=batch_size,class_mode='categorical',shuffle=True)
val_gen=val_data_gen.flow_from_directory(validate,color_mode='grayscale',target_size=(rows,cols),batch_size=batch_size,class_mode='categorical',shuffle=True)

Found 28911 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [ ]:
from scipy import misc
from scipy import ndimage
blurred_f = ndimage.gaussian_filter(train_gen, 0.3)
filter_blurred_f = ndimage.gaussian_filter(blurred_f, 0.2)
alpha = 4
train_gen = blurred_f + alpha * (blurred_f - filter_blurred_f)

In [6]:
model=Sequential()

In [7]:
model.add(Conv2D(32,(2,2),padding='same',kernel_initializer='he_normal',input_shape=(rows,cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(2,2),padding='same',kernel_initializer='he_normal',input_shape=(rows,cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [8]:
model.add(Conv2D(64,(2,2),padding='same',kernel_initializer='he_normal',input_shape=(rows,cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(2,2),padding='same',kernel_initializer='he_normal',input_shape=(rows,cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [9]:
model.add(Conv2D(128,(2,2),padding='same',kernel_initializer='he_normal',input_shape=(rows,cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(2,2),padding='same',kernel_initializer='he_normal',input_shape=(rows,cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [10]:
model.add(Conv2D(256,(2,2),padding='same',kernel_initializer='he_normal',input_shape=(rows,cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(2,2),padding='same',kernel_initializer='he_normal',input_shape=(rows,cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [11]:
model.add(Flatten())
model.add(Dense(256,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

In [12]:
model.add(Flatten())
model.add(Dense(512,kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

In [13]:
model.add(Dense(n,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [14]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

In [15]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [16]:
checkpoint= ModelCheckpoint(r'C:\Users\majaa\data\Emotion.h5',monitor='val_accuracy',mode='max',save_best_only=True,verbose=1)
earlystop=EarlyStopping(monitor='val_accuracy',min_delta=0,patience=10,verbose=1,restore_best_weights=True)
reducelr=ReduceLROnPlateau(monitor='val_accuracy',factor=0.2,patience=3,verbose=1,min_delta=0.0001)

In [17]:
callbacks=[checkpoint,earlystop,reducelr]

In [18]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [48]:
train_sample=28911
val_sample=7066
epochs=100
his=model.fit_generator(train_gen,steps_per_epoch=train_sample//batch_size,epochs=epochs,callbacks=callbacks,validation_data=val_gen,validation_steps=val_sample//batch_size)

Epoch 1/100
  2/275 [..............................] - ETA: 4:22 - loss: 1.0323 - accuracy: 0.6095 

KeyboardInterrupt: 